In [1]:
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import warnings
from tqdm import tqdm
warnings.filterwarnings("ignore", category=UserWarning)

In [2]:
src_test_parquet = 'ke_test_data/test.parquet'

In [3]:
test = pq.read_table(src_test_parquet).to_pandas()

In [4]:
# загружаем результаты обучения
from joblib import load

DG = load('DG.pkl')
model_params = load('model_params.pkl')
catalogs = load('catalogs.pkl')
cat2id = load('cat2id.pkl')
train_catalogs = load('train_catalogs.pkl')
train_cat2id = load('train_cat2id.pkl')

In [5]:
def all_predcestors(g, id, max_len=7):
    # циклично запрашиваем предков, пока их не будет совсем
    all_preds = [id]
    while True:
        preds = list(g.predecessors(id))
        if len(preds) == 0:
            break
        all_preds.append(preds[0])
        id = all_preds[-1]

    # доводим до максимального размера, чтобы вмещалось в tensor/array
    # паддинг -1 (0 занят нашим каталогом, мы позже их закодируем, но всеравно...)
    if len(all_preds) < max_len:
        all_preds = [-1] * (max_len - len(all_preds)) + all_preds
    return all_preds[::-1]

In [6]:
def hist_edges_mapper(val, bin_borders=model_params['hist_bin_edges']):
    # мэпим значения в диапазоны гистограммы
    resval = []
    for i in range(len(bin_borders)-1):
        if bin_borders[i] <= val < bin_borders[i+1]:
            resval.append(1)
        elif i == len(bin_borders) - 2:
            if val == bin_borders[i+1]:
                resval.append(1)
            else:
                resval.append(0)
        else:
            resval.append(0)

    global num_pbar
    num_pbar.update(1)

    return resval

def get_text_data_func(x, sep=' '):
    # дергает текстовые данные из входящего датафрейма
    result = x.copy()
    result['text'] = ''
    for i, key in enumerate(model_params['text']):
        if i != 0:
            result['text'] = result['text'] + sep
        result['text'] = result['text'] + result[key].astype(str)

    return result['text'].to_numpy()

def get_numeric_data_func(x):
    # дергает числовые данные из входящего датафрейма
    x = x[model_params['numeric']].copy()
    column = model_params['numeric'][0]
    x[column] = x[column].apply(hist_edges_mapper)
    return pd.DataFrame(x['rating'].to_list()).to_numpy()

In [7]:
# прогрессбары внутри пайплайна
num_pbar = None
text_pbar = None

def start_num_pbar(x):
    global num_pbar
    num_pbar = tqdm(total=len(x), desc='Nums processing')
    return x


def shut_num_pbar(x):
    global num_pbar
    num_pbar.close()
    return x


def start_text_pbar(x):
    global text_pbar
    text_pbar = tqdm(total=len(x), desc='Text processing')
    return x


def shut_text_pbar(x):
    global text_pbar
    text_pbar.close()
    return x

In [8]:
# бинарные кодеры\декодеры
def dummify_labels(df_in, column, column_dict, catalogs):
    """
    разбивает столбец датафрейма на бинарные столбцы из списка. Просто принцип onehot
    """
    y = df_in[column].copy()
    # ушли в "примитивы", поиск по пандасу значительно дольше.
    out = np.zeros((len(y.index), len(catalogs)), dtype=float)

    for i, cat in enumerate(y[column[0]].values):
        cat_position = column_dict[cat]
        out[i][cat_position] = 1

    out = pd.DataFrame(out, index=y.index, columns=catalogs).astype(int)

    return out

def undummify_labels(df_dimmies, column, column_list):
    """
    возвращает бинарные столбцы таргета в исходный один столбец.
    """

    y = df_dimmies.to_numpy()
    out = np.argmax(y, axis=1)
    out = np.array([column_list[i] for i in out])

    return pd.DataFrame(out, columns=column, index=df_dimmies.index)

In [9]:
# свой токенайзер + стэммер  (так мы уменьшим размерность с ~80к до ~50к уникальных токенов
import nltk
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
import re

regex = r"[\s\|$:*!?#:,.(){}\"\'\$\^\/\\\[\]\-+”“%¡¿&;«»`\d]"

punctuation = set(['\t','\n','\\', '', 'none'])

nltk.download("stopwords")
nltk.download('punkt')

russian_stopwords = set(stopwords.words("russian"))  # список стоп слов


class StemTokenizer:
    # Стэммер вычленяет корни слов
    def __init__(self):
        self.stemer = SnowballStemmer('russian')
    def __call__(self, doc):
        # 1) делаем список токенов через regexp
        # 2) делаем стэмминг корней слов
        # 3) фильтруем полученные леммы через стоп слова
        # 4) возвращаем список
        regex_num_ponctuation = punctuation

        resval = [self.stemer.stem(t) for t in re.split(regex, doc.lower())
                if t not in regex_num_ponctuation and len(t) >= model_params['t_min_len']]

        global text_pbar
        text_pbar.update(1)

        return resval


from pymystem3 import Mystem

class LemmaTokenizer():
    # Лемматайзер приводит слова к неопределенному виду (сохраняется некоторый контекст)
    # почти в два раза медленнее и на ~0,001 дает точности больше чем стэмер
    # не пиклится :(
    def __init__(self):
        self.mystem = Mystem()

    def __call__(self, doc):
        # 1) делаем список токенов через regexp
        # 2) делаем лемминг слов
        # 3) фильтруем полученные леммы через стоп слова
        # 4) возвращаем список

        resval = []
        for t in re.split(regex, doc.lower()):
            lemma = self.mystem.lemmatize(t)  # позвращает список лемм
            # tokens.extend(lemma)
            for tt in lemma:
                if tt not in punctuation and len(tt) >= model_params['t_min_len']:
                    resval.append(tt)

        global text_pbar
        text_pbar.update(1)

        return resval

[nltk_data] Downloading package stopwords to /home/kit-
[nltk_data]     kat/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/kit-kat/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [10]:
# загружаем обученную модель
from joblib import load
pl = load('pl.pkl')

In [11]:
X_test = test[model_params['text'] + model_params['numeric']]

In [12]:
result = pl.predict_proba(X=X_test)

Text processing: 100%|██████████| 70864/70864 [00:32<00:00, 2201.38it/s]


In [13]:
result = pd.DataFrame(result, index=X_test.index, columns=train_catalogs)
result = undummify_labels(result, model_params['labels'], train_catalogs)

In [14]:
test = test[['id']]
test['predicted_category_id'] = result[model_params['labels'][0]]

In [15]:
test.head()

,id,predicted_category_id
0,1070974,11574
1,450413,11878
2,126857,13299
3,1577569,13061
4,869328,12813


In [16]:
table = pa.Table.from_pandas(test)
pq.write_table(table, 'result.parquet')

In [17]:
exit(0)